In [1]:
import numpy as np
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import euclidean

In [2]:
class Kmeans:
    '''
    K-means is a clustering algorithm that finds convex clusters. 
    
    The user specifies the number of clusters a priori.'''
    
    
    def __init__(self, K=2, init='k++', random_state=42):
        self.K_ = K
        self.init_ = init
        self._seed = random_state
        self.centroid_init_ = None
        self.centroid_loc_ = None
        self._centroid_test = None
        self._nrows = None
        self._nfeatures = None
        self.count_ = None
        self.inertia_ = None
        self.silhouette_ = None

    
    def _k_plus_plus(self, X, k):
        '''k++ implementation for cluster initialization

        Input:
            X = numpy data matrix
            k = number of centroids
        Output:
            k++ selected centroid indices
        '''
        n_clust = 0
        idx = np.arange(len(X))

        while n_clust < k:
            # initialize 
            if n_clust == 0:
                choice = np.random.choice(idx, size=1)
                cluster_idx = np.array(choice)
            else:
                distances = np.array([euclidean(X[choice], X[i]) for i in range(len(X))])
                for i,_ in enumerate(distances):
                    if i in cluster_idx:
                        distances[i] = 0
                total_distance = np.sum(distances)
                prob = np.array([distance/total_distance for distance in distances])    
                choice = np.random.choice(idx, size=1, p=prob)
                if choice not in cluster_idx:
                    cluster_idx = np.append(cluster_idx, choice)
                    np.delete(idx, choice)
            n_clust = len(cluster_idx)
        return cluster_idx
    
    
    def _initialize_centroids(self, X, seed=True):
        '''Randomly initialize centroids.
        
        Input:
            X = numpy data matrix
        Output:
            K centroid locations
        '''
        if seed == True:
            np.random.seed(self._seed)
        
        self._nrows = X.shape[0]
        self._nfeatures = X.shape[1]
        
        assert self.init_ == 'random' or 'k++' or 'forgy', "choose 'random', 'k++', 'forgy' for init"
        
        if self.init_ == 'random':
            centroid_locs = [np.random.randint(low=np.min(X),
                                               high=np.max(X),
                                               size= self._nfeatures) for _ in range(self.K_)]
        elif self.init_ == 'k++':
            centroid_locs = X[self._k_plus_plus(X, self.K_)]

            self.centroid_loc_ = centroid_locs
        elif self.init_ == 'forgy':
            centroid_locs = X[np.random.choice(self._nrows, replace=False, size=self.K_)]
        
        self.centroid_loc_ = centroid_locs
        self.centroid_init_ = np.array(centroid_locs).reshape(self.K_,-1)

    
    def _calc_distance(self, X):
        '''Calculate the distance between data points and centroids.
        
        Input:
            X = numpy data matrix
        Output:
            matrix of distance between each data point and each cluster
        '''
        return np.array([euclidean(X[i], self.centroid_loc_[j]) 
                         for i in range(self._nrows) 
                         for j in range(self.K_)]).reshape(self._nrows, self.K_)
    
  
    def _update_cluster_loc(self, X):
        '''Update centroid locations for each iteration of fitting.
        
        Input:
            X = numpy data matrix
        Output:
            updated centroid location
        '''
        predictions = self.predict(X)
        idx = set(predictions)
        assert idx != self.K_, "Bad initialization: use 'k++' or 'forgy' init"
        self.centroid_loc_ = np.array([np.mean(X[self.predict(X) == i], axis=0)
                                           for i in range(self.K_)]).reshape(len(idx),-1)

        
    def fit(self, X):
        '''Calculate centroid positions given training data.
        
        Input:
            X = numpy data matrix
        Output:
            fitted centroid locations
        '''
        self._initialize_centroids(X, seed=True)
        self.count_ = 0
        while True:
            self.count_ += 1
            self._centroid_test = self.centroid_loc_
            self._update_cluster_loc(X)

            if np.all(self._centroid_test == self.centroid_loc_):
                self._inertia()
                self._silhouette_score()
                break
    
    
    def predict(self, X):
        '''Assign data points to cluster number.
        
        Input:
            X = numpy data matrix
        Output:
            cluster ID
        '''
        return np.argmin(self._calc_distance(X), axis=1)
    
    
    def _inertia(self):
        '''Calculates the total inertia after fitting.'''
        self.inertia_ = np.sum([euclidean(X[self.predict(X)==j][i], self.centroid_loc_[j])**2
                                for j in range(self.K_)
                                for i in range(X[self.predict(X)==j].shape[0])])
    
    
    # Need to write this part from scratch
    def _silhouette_score(self):
        '''Calculates the silhouette score after fitting.'''
        self.silhouette_ = silhouette_score(X, self.predict(X))
        
    # Extensions
    # 1. Add multiple initializations to find better clusters
    # 2. Create plot methods 

---

In [3]:
X = np.random.randint(0,30,90).reshape(30,3).astype('float')
X[10,:]

array([ 8.,  7.,  9.])

In [21]:
km = Kmeans(K=3, random_state=33, init='forgy')
km.fit(X)

In [22]:
km.centroid_init_

array([[ 28.,   0.,   0.],
       [  8.,   4.,   1.],
       [ 20.,  12.,   4.]])

In [23]:
km.centroid_loc_

array([[ 28.        ,   0.        ,   0.        ],
       [  6.21428571,   8.28571429,   8.14285714],
       [ 18.66666667,  15.46666667,  14.53333333]])

In [24]:
km.predict(X)

array([1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1,
       1, 1, 2, 1, 1, 1, 0])

In [25]:
km.count_

2

In [26]:
km.inertia_

3115.7285714285717

In [27]:
km.silhouette_

0.25566979262344464

In [38]:
d = [(i,j, euclidean(km.centroid_loc_[i], km.centroid_loc_[j]))
     for i in range(km.K_) for j in range(km.K_) if i!=j]
d

[(0, 1, 24.68960369492076),
 (0, 2, 23.18505265611158),
 (1, 0, 24.68960369492076),
 (1, 2, 15.731053824260112),
 (2, 0, 23.18505265611158),
 (2, 1, 15.731053824260112)]

In [84]:
for i, tup in enumerate(d):
    for j in range(1,km.K_):
        print(i,j, tup)

0 1 (0, 1, 24.68960369492076)
0 2 (0, 1, 24.68960369492076)
1 1 (0, 2, 23.18505265611158)
1 2 (0, 2, 23.18505265611158)
2 1 (1, 0, 24.68960369492076)
2 2 (1, 0, 24.68960369492076)
3 1 (1, 2, 15.731053824260112)
3 2 (1, 2, 15.731053824260112)
4 1 (2, 0, 23.18505265611158)
4 2 (2, 0, 23.18505265611158)
5 1 (2, 1, 15.731053824260112)
5 2 (2, 1, 15.731053824260112)


In [35]:
euclidean(km.centroid_loc_[2], km.centroid_loc_[0])

23.18505265611158

---

In [11]:
km2 = Kmeans(random_state=33, init='k++')
km2.fit(X)

In [12]:
km2.centroid_init_

array([[ 19.,   7.,  15.],
       [  0.,  18.,  29.]])

In [13]:
km2.centroid_loc_

array([[ 15.17647059,  17.35294118,   9.23529412],
       [  6.46153846,  15.53846154,  23.38461538]])

In [14]:
km2.predict(X)

array([0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0])

In [15]:
km2.count_

4

In [16]:
km2.inertia_

4346.950226244343

In [17]:
km2.silhouette_

0.2706598718853046

---

In [18]:
from sklearn.cluster import KMeans

In [19]:
kmeans = KMeans(n_clusters=2, init='k-means++')
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [20]:
kmeans.cluster_centers_

array([[  6.46153846,  15.53846154,  23.38461538],
       [ 15.17647059,  17.35294118,   9.23529412]])

In [21]:
kmeans.predict(X)

array([1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1], dtype=int32)

In [22]:
kmeans.n_iter_

3

In [23]:
kmeans.inertia_

4346.9502262443439

In [24]:
silhouette_score(X, kmeans.predict(X))

0.2706598718853046